## How to export thousands of image chips from Earth Engine in a few minutes

This source code of this notebook was adopted from the Medium post - [Fast(er) Downloads](https://gorelick.medium.com/fast-er-downloads-a2abd512aa26) by Noel Gorelick. Credits to Noel.  

Due to the [limitation](https://docs.python.org/3/library/multiprocessing.html) of the [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) package, the functionality of this notebook can only be run in the top-level. Therefore, it could not be implemented as a function under geemap.

### Install packages

Uncomment the following line to install the required packages.

In [ ]:
pip install geemap

### Import libraries

In [2]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

### Initialize GEE to use the high-volume endpoint

- [high-volume endpoint](https://developers.google.com/earth-engine/cloud/highvolume)

In [6]:
ee.Initialize

<function ee.Initialize(credentials='persistent', opt_url=None, cloud_api_key=None, http_transport=None, project=None)>

In [4]:
pip install ee

  Preparing metadata (setup.py) ... done
  Created wheel for ee: filename=ee-0.2-py3-none-any.whl size=3655 sha256=a815e7c1cf37e75ff10e042f2ca1e7dce765d07802e8f29109e466385435ec6e
  Stored in directory: /root/.cache/pip/wheels/cb/b1/a5/d16d395d190f232181add4c4af67a74c2de272875759921aef
Successfully built ee


### Create an interactive map

In [21]:
Map = geemap.Map()
import geemap

Map = geemap.Map()

imgVV = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select('VV')

def func_wzh(image):
          edge = image.lt(-30.0)
          maskedImage = image.mask().And(edge.Not())
          return image.updateMask(maskedImage) \
        .map(func_wzh)

desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spAugL = ee.Filter.date('2020-08-30', '2020-09-15')

spImgAugL = asc.filter(spAugL).mean()

Map.setCenter(5.9, 4.3, 12)
Map.addLayer(spImgAugL, {'min': -25, 'max': 5}, 'Late August Spill', True)

Map

Map(center=[4.3, 5.9], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Define the Region of Interest (ROI)

You can use the drawing tools on the map to draw an ROI, then you can use `Map.user_roi` to retrieve the geometry. Alternatively, you can define the ROI as an ee.Geometry as shown below.

In [23]:
# region = Map.user_roi
region = ee.Geometry.Polygon(
    [
        [
[5.909271, 4.270067],
   [6.006432, 4.270067],
   [6.006432, 4.370374],
   [5.909271, 4.370374],
   [5.909271, 4.270067]]
    ],
    None,
    False,
)

In [22]:
region = Map.user_roi
region.getInfo()

{'type': 'Polygon',
 'coordinates': [[[5.909271, 4.270067],
   [6.006432, 4.270067],
   [6.006432, 4.370374],
   [5.909271, 4.370374],
   [5.909271, 4.270067]]]}

### Define the image source

Using the 1-m [NAIP imagery](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ).

In [24]:
image = (ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select('VV')
)

Using the 10-m [Sentinel-2 imagery](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands).

In [ ]:
# image = ee.ImageCollection('COPERNICUS/S2_SR') \
#             .filterBounds(region) \
#             .filterDate('2021', '2022') \
#             .select('B8', 'B4', 'B3') \
#             .median() \
#             .visualize(min=0, max=4000) \
#             .clip(region)

### Set parameters

If you want the exported images to have coordinate system, change `format` to `GEO_TIFF`. Otherwise, you can use `png` or `jpg` formats.

In [25]:
params = {
    'count': 50,  # How many image chips to export
    'buffer': 127,  # The buffer distance (m) around each point
    'scale': 100,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': 'tile_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': '.',  # The output directory. Default to the current working directly
}

### Add layers to map

In [26]:
Map.addLayer(image, {}, "Image")
Map.addLayer(region, {}, "ROI", False)
Map.setCenter(5.9458205,4.323315, 10)
Map

Map(bottom=512001.0, center=[4.323315, 5.9458205], controls=(WidgetControl(options=['position', 'transparent_b…

### Generate a list of work items

In the example, we are going to generate 1000 points using the stratified random sampling, which requires a `classBand`. It is the name of the band containing the classes to use for stratification. If unspecified, the first band of the input image is used. Therefore, we have toADD a new band with a constant value (e.g., 1) to the image. The result of the `getRequests()`function returns a list of dictionaries containing points.

In [27]:
def getRequests():
    img = ee.Image(1).rename("Class").addBands(image)
    points = img.stratifiedSample(
        numPoints=params['count'],
        region=region,
        scale=params['scale'],
        seed=params['seed'],
        geometries=True,
    )
    Map.data = points
    return points.aggregate_array('.geo').getInfo()

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Create a function for downloading image

The `getResult()` function then takes one of those points and generates an image centered on that location, which is then downloaded as a PNG and saved to a file. This function uses `image.getThumbURL()` to select the pixels, however you could also use `image.getDownloadURL()` if you wanted the output to be in GeoTIFF or NumPy format ([source](https://gorelick.medium.com/fast-er-downloads-a2abd512aa26)).

In [32]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    point = ee.Geometry.Point(point['coordinates'])
    region = point.buffer(params['buffer']).bounds()

    if params['format'] in ['png', 'jpg']:
        url = image.getThumbURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )

    if params['format'] == "GEO_TIFF":
        ext = 'tif'
    else:
        ext = params['format']

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()
# Your code to generate the filename and open the file
    # Generate filename
    basename = str(index).zfill(len(str(params['count'])))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"

# Specify the path to your Google Drive folder
drive_folder = '/content/drive/MyDrive/OilSpill_Dataset'

# Construct the full path to save the file in your Google Drive
drive_filename = os.path.join(drive_folder, filename)

# Open the file in Google Drive and save it
with open(drive_filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)

print("Done: ", basename, "saved in Google Drive at:", drive_filename)

NameError: ignored

### Download images

In [ ]:
%%time
logging.basicConfig()
items = getRequests()

pool = multiprocessing.Pool(params['processes'])
pool.starmap(getResult, enumerate(items))

pool.close()

### Retrieve sample points

In [ ]:
Map.addLayer(Map.data, {}, "Sample points")
Map